In [22]:
import re
import logging
import numpy as np
import pandas as pd
from config import config
from get_offers_by_city import connect

In [23]:
conn = connect(config())

query = '''
    SELECT * FROM rent_infos_cleaned
'''

cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetchall()
conn.close()

Connecting to database...
Connection successful


In [24]:
df = pd.DataFrame(result, columns=['offer_id', 'extraction_date', 'lat', 'lng', 'area_m2', 
                                   'furnished','zip_code', 'objectcat', 'rooms', 'build_year', 
                                   'state', 'city','sub_category', 'balcony', 'heat_type', 
                                   'offer_title', 'kitchen','rent_price', 'garden'])

In [25]:
df.head()

,offer_id,extraction_date,lat,lng,area_m2,furnished,zip_code,objectcat,rooms,build_year,state,city,sub_category,balcony,heat_type,offer_title,kitchen,rent_price,garden
0,45678593,2021-11-03 05:14:31,50.101680,8.658270,65.0,1,60327.0,Wohnung,3.0,1903.0,Hessen,Frankfurt am Main,Etagenwohnung,0,Zentralheizung,* frisch renovierte 3 Zimmerwohnung * Innensta...,0,790.0,0
1,45776903,2021-11-03 05:14:31,51.048430,13.750650,40.0,1,1069.0,Wohnung,2.0,2021.0,Sachsen,Dresden,Etagenwohnung,0,NaN,Zwischen Elbe und Großem Garten - Neubau 2-Zim...,0,420.0,0
2,44924940,2021-11-03 05:14:31,51.044313,13.699018,55.0,1,1159.0,Wohnung,2.0,NaN,Sachsen,Dresden,Etagenwohnung,0,Zentralheizung,Frisch sanierte 2 Raum Wohnung in ruhiger Lage...,0,440.0,0
3,44630030,2021-11-03 05:14:31,50.111310,8.703920,50.0,1,60314.0,Wohnung,2.0,1955.0,Hessen,Frankfurt am Main,Wohnung,0,Zentralheizung,wunderschön möbilierte Wohnung gegenüber ECB,1,1299.0,0
4,45514780,2021-11-03 05:14:31,51.499150,7.451470,85.7,1,44139.0,Wohnung,3.0,2021.0,Nordrhein-Westfalen,Dortmund,Etagenwohnung,1,NaN,Erstaunlich gut wohnen! VIVAWEST lässt Ihnen d...,0,935.0,0


In [21]:
df.isna().sum()

offer_id             0
extraction_date      0
lat                  0
lng                  0
area_m2            487
furnished            0
zip_code             1
objectcat            0
rooms               65
build_year           0
state                0
city                 0
sub_category         0
balcony              0
heat_type            0
offer_title          0
kitchen              0
rent_price          19
garden               0
dtype: int64

In [10]:
df['kitchen'].unique()

array([Decimal('NaN'), Decimal('NaN'), Decimal('NaN'), ...,
       Decimal('NaN'), Decimal('NaN'), Decimal('NaN')], dtype=object)